In [24]:
import numpy as np
import pandas as pd
from characters.sparkle import Sparkle
from characters.qingque import Qingque
from characters.silver_wolf import Silver_Wolf
from characters.fu_xuan import Fu_Xuan
from simulations.qingque_mono_quantum import QQ_Mono_Quantum, Initialize_QQ_Mono_Quantum
from utils.arena import Arena
from utils.data_table import Action_Table, Qingque_Probability_Table, Battle_Buffs_Tracker

# Character Imports

In [25]:
qingque_speed = 98

In [26]:
silver_wolf_speed = 161

In [27]:
sparkle_speed = 300

In [28]:
fu_xuan_speed = 134
fu_xuan_technique = True

# Simulations

In [29]:
arena, qq, spk, sw, fx = Initialize_QQ_Mono_Quantum(qingque_speed=qingque_speed, fu_xuan_speed=fu_xuan_speed, sparkle_speed=sparkle_speed, silver_wolf_speed=silver_wolf_speed)

In [30]:
action_dataframe = Action_Table()
probability_dataframe = Qingque_Probability_Table()
buffs_df = Battle_Buffs_Tracker()

In [31]:
QQ_Mono_Quantum(arena, qq, spk, sw, fx, action_df=action_dataframe, prob_df= probability_dataframe, buffs_df=buffs_df)

Empty DataFrame
Columns: [Name, Holder Name, Source Name, Recipient Name, Stat, Amount, Remaining Turns]
Index: []
Empty DataFrame
Columns: [Name, Holder Name, Source Name, Recipient Name, Stat, Amount, Remaining Turns]
Index: []
Empty DataFrame
Columns: [Name, Holder Name, Source Name, Recipient Name, Stat, Amount, Remaining Turns]
Index: []
Applying Cipher on Qingque
Applying Cipher on Sparkle
Applying Cipher on Silver Wolf
Applying Cipher on Fu Xuan
     Name Holder Name Source Name Recipient Name  Stat Amount Remaining Turns
0  Cipher     Sparkle     Sparkle        Sparkle  Dmg%     30               1
Empty DataFrame
Columns: [Name, Holder Name, Source Name, Recipient Name, Stat, Amount, Remaining Turns]
Index: []


c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)
c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)
c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulato

Empty DataFrame
Columns: [Name, Holder Name, Source Name, Recipient Name, Stat, Amount, Remaining Turns]
Index: []
Applying Cipher on Qingque
Applying Cipher on Sparkle
Applying Cipher on Silver Wolf
Refreshing Cipher on Fu Xuan
     Name Holder Name Source Name Recipient Name  Stat Amount Remaining Turns
0  Cipher     Sparkle     Sparkle        Sparkle  Dmg%     30               1
Empty DataFrame
Columns: [Name, Holder Name, Source Name, Recipient Name, Stat, Amount, Remaining Turns]
Index: []
Empty DataFrame
Columns: [Name, Holder Name, Source Name, Recipient Name, Stat, Amount, Remaining Turns]
Index: []
Applying Cipher on Qingque
Applying Cipher on Sparkle
Refreshing Cipher on Silver Wolf
Applying Cipher on Fu Xuan
     Name Holder Name Source Name Recipient Name  Stat Amount Remaining Turns
0  Cipher     Sparkle     Sparkle        Sparkle  Dmg%     30               1
Empty DataFrame
Columns: [Name, Holder Name, Source Name, Recipient Name, Stat, Amount, Remaining Turns]
Index: []


# Results

## Action List

In [32]:
pd.set_option("display.max_rows", None)
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Sparkle,-1.0,33.333333,6,55.00,Skill,-1
1,Qingque,0.0,51.120408,6,70.00,Enhanced Basic,-3
2,Silver Wolf,0.0,62.111801,3,55.00,Skill,-1
3,Sparkle,0.0,66.666667,2,90.82,Skill,-1
4,Fu Xuan,0.0,74.626866,1,67.50,Skill,-1
5,Sparkle,0.0,100.000000,0,126.64,Ultimate,4
6,Sparkle,0.0,100.000000,4,5.00,Skill,-1
7,Qingque,0.0,100.100000,3,70.00,Enhanced Basic,-2
8,Silver Wolf,0.0,124.223602,1,55.00,Basic,1
9,Sparkle,0.0,133.333333,2,40.82,Skill,-1


# Buffs List

In [33]:
pd.set_option("display.max_rows", None)
#buffs_df.df
buffs_df.df

,Cycle,Action Gauge,Name,Holder Name,Source Name,Recipient Name,Stat,Amount,Remaining Turns
0,0.0,51.120408,Dreamdiver,Qingque,Sparkle,Qingque,Crit Dmg,100,1
1,0.0,100.000000,Cipher,Sparkle,Sparkle,Sparkle,Dmg%,30,2
2,0.0,100.100000,Dreamdiver,Qingque,Sparkle,Qingque,Crit Dmg,100,1
3,0.0,100.100000,Cipher,Qingque,Sparkle,Qingque,Dmg%,30,2
4,0.0,100.100000,Dreamdiver,Qingque,Sparkle,Qingque,Crit Dmg,100,1
5,0.0,124.223602,Cipher,Silver Wolf,Sparkle,Silver Wolf,Dmg%,30,2
6,0.0,133.333333,Cipher,Sparkle,Sparkle,Sparkle,Dmg%,30,1
7,0.0,149.253731,Cipher,Fu Xuan,Sparkle,Fu Xuan,Dmg%,30,2
8,1.0,151.220408,Cipher,Qingque,Sparkle,Qingque,Dmg%,30,1
9,1.0,151.220408,Dreamdiver,Qingque,Sparkle,Qingque,Crit Dmg,100,1


## QQ Probability List

In [34]:
probability_dataframe_styled = probability_dataframe.df.style.set_caption(f"Qingque speed = {qingque_speed}, Sparkle speed = {sparkle_speed}, Silver Wolf Speed = {silver_wolf_speed}, Fu Xuan speed = {fu_xuan_speed}")
probability_dataframe_styled

,Cycle,Action Gauge,Current SP,Current Tiles,Hidden Hand,P(Low),P(High),QQ Draws,QQ Roll,SP Used
0,0.000000,51.120408,6,2,True,64.791952,83.894731,4,0.705712,3
1,0.000000,100.100000,3,5,True,76.070720,89.212348,3,0.883848,2
2,1.000000,151.220408,2,4,True,0.000000,30.041152,1,0.006587,0
3,1.000000,200.100000,3,4,True,0.000000,30.041152,1,0.063625,0
4,2.000000,251.220408,4,4,True,0.000000,30.041152,1,0.041067,0
5,2.000000,300.100000,5,4,True,0.000000,30.041152,1,0.138145,0
6,3.000000,351.220408,5,3,True,49.245541,76.070720,3,0.599329,2
7,3.000000,400.100000,3,5,True,0.000000,49.245541,1,0.184805,0
8,4.000000,451.220408,4,4,True,30.041152,64.791952,2,0.319346,1
9,4.000000,500.100000,4,4,True,0.000000,30.041152,1,0.016854,0


# Looped Simulation

In [35]:
df = pd.merge(action_dataframe.df, buffs_df.df, on="Action Gauge", how="outer")
df = df.where(pd.notnull(df), None)
df = df.drop(columns=["Holder Name", "Cycle_y"])
df = df.rename(columns={"Name_x":"Name", "Cycle_x":"Cycle","Name_y":"Buff Name" })

In [37]:
df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain,Buff Name,Source Name,Recipient Name,Stat,Amount,Remaining Turns
0,Sparkle,-1.0,33.333333,6,55.00,Skill,-1,None,None,None,None,None,None
1,Qingque,0.0,51.120408,6,70.00,Enhanced Basic,-3,Dreamdiver,Sparkle,Qingque,Crit Dmg,100,1
2,Silver Wolf,0.0,62.111801,3,55.00,Skill,-1,None,None,None,None,None,None
3,Sparkle,0.0,66.666667,2,90.82,Skill,-1,None,None,None,None,None,None
4,Fu Xuan,0.0,74.626866,1,67.50,Skill,-1,None,None,None,None,None,None
5,Sparkle,0.0,100.000000,0,126.64,Ultimate,4,Cipher,Sparkle,Sparkle,Dmg%,30,2
6,Sparkle,0.0,100.000000,4,5.00,Skill,-1,Cipher,Sparkle,Sparkle,Dmg%,30,2
7,Qingque,0.0,100.100000,3,70.00,Enhanced Basic,-2,Dreamdiver,Sparkle,Qingque,Crit Dmg,100,1
8,Qingque,0.0,100.100000,3,70.00,Enhanced Basic,-2,Cipher,Sparkle,Qingque,Dmg%,30,2
9,Qingque,0.0,100.100000,3,70.00,Enhanced Basic,-2,Dreamdiver,Sparkle,Qingque,Crit Dmg,100,1
